In [91]:
import pandas as pd
import numpy as np
from datetime import datetime, time

In [92]:
df = pd.read_csv('Coffee_sales.csv')

In [94]:
month_weights = [5, 1, 8, 3, 8, 12, 12, 2, 8, 5, 3, 13]
month_probs = np.array(month_weights) / np.sum(month_weights)

all_dates = pd.date_range(start="2023-01-01", end="2025-12-31", freq="D")
weights_by_month = {i+1: w for i, w in enumerate(month_weights)}
date_weights = np.array([weights_by_month[d.month] for d in all_dates])
date_probs = date_weights / date_weights.sum()

In [95]:
selected_dates = np.random.choice(all_dates, size=n, p=date_probs)

# Corrigir a geração de horários
times = [datetime.time(datetime.combine(selected_dates[i], 
                                       datetime.min.time().replace(hour=np.random.randint(7, 22)))) 
        for i in range(n)]

TypeError: combine() argument 1 must be datetime.date, not numpy.datetime64

In [79]:
cash_types = ["Cash", "Credit Card", "Debit Card", "Mobile Payment"]
coffee_names = ["Espresso", "Latte", "Cappuccino", "Mocha", "Americano", "Macchiato"]

In [80]:
n = 25279
dates = pd.Series(np.random.choice(date_range, n))   # <<< vira Series do pandas
times = pd.to_datetime(np.random.randint(0, 24*60*60, n), unit="s").time

In [81]:
coffee_weights = [30, 25, 20, 10, 10, 5]   # proporção
coffee_probs = np.array(coffee_weights) / np.sum(coffee_weights)

In [82]:
synthetic_data = pd.DataFrame({
    "hour_of_day": [t.hour for t in times],
    "cash_type": np.random.choice(cash_types, n, p=[0.5, 0.2, 0.2, 0.1]),
    "money": np.round(np.random.uniform(2.5, 15.0, n), 2),
    "coffee_name": np.random.choice(coffee_names, n, p=coffee_probs),
    "Time_of_Day": ["Morning" if 5 <= t.hour < 12 else
                    "Afternoon" if 12 <= t.hour < 18 else
                    "Night"
                    for t in times],
    "Weekday": pd.Series(selected_dates).dt.day_name(),
    "Month_name": pd.Series(selected_dates).dt.month_name(),
    "Weekdaysort": pd.Series(selected_dates).dt.weekday,
    "Monthsort": pd.Series(selected_dates).dt.month,
    "Date": pd.Series(selected_dates).dt.date,
    "Time": times,
    "week_number": pd.Series(selected_dates).dt.isocalendar().week,
    "day_of_month": pd.Series(selected_dates).dt.day
})

In [83]:
print(df)

       hour_of_day       cash_type  money coffee_name Time_of_Day    Weekday  \
0                2     Credit Card   4.76       Latte       Night  Wednesday   
1                9  Mobile Payment   4.94   Americano     Morning  Wednesday   
2               21            Cash   6.13       Latte       Night     Monday   
3                2            Cash   8.06    Espresso       Night     Sunday   
4                5     Credit Card  11.27   Americano     Morning   Saturday   
...            ...             ...    ...         ...         ...        ...   
25274            7            Cash   3.58       Latte     Morning   Saturday   
25275           19     Credit Card  10.35   Macchiato       Night     Sunday   
25276            1     Credit Card  11.12       Latte       Night     Monday   
25277           13      Debit Card   3.03       Latte   Afternoon     Sunday   
25278            2            Cash   3.79       Latte       Night     Monday   

      Month_name  Weekdaysort  Monthsor

In [84]:
df['cash_type'].value_counts()

Cash              12520
Credit Card        5137
Debit Card         5129
Mobile Payment     2493
Name: cash_type, dtype: int64

In [107]:
n = 37162

In [108]:
cash_types = ["Cash", "Credit Card", "Debit Card", "Mobile Payment"]
coffee_names = ["Espresso", "Latte", "Cappuccino", "Mocha", "Americano", "Macchiato"]
coffee_weights = [30, 25, 20, 10, 10, 5]
coffee_probs = np.array(coffee_weights) / np.sum(coffee_weights)

month_weights = [5, 1, 8, 3, 8, 12, 12, 2, 8, 5, 3, 13]
month_probs = np.array(month_weights) / np.sum(month_weights)

all_dates = pd.date_range(start="2023-01-01", end="2025-12-31", freq="D")
weights_by_month = {i+1: w for i, w in enumerate(month_weights)}
date_weights = np.array([weights_by_month[d.month] for d in all_dates])
date_probs = date_weights / date_weights.sum()

selected_dates = np.random.choice(all_dates, size=n, p=date_probs)

# Gerar horários aleatórios
times = [time(hour=np.random.randint(7, 22)) for _ in range(n)]

In [109]:
synthetic_data = pd.DataFrame({
    "hour_of_day": [t.hour for t in times],
    "cash_type": np.random.choice(cash_types, n, p=[0.15, 0.45, 0.30, 0.10]),  # Ajuste europeu: mais cartão
    "money": np.round(np.random.uniform(2.5, 15.0, n), 2),
    "coffee_name": np.random.choice(coffee_names, n, p=coffee_probs),
    "Time_of_Day": ["Morning" if 5 <= t.hour < 12 else
                    "Afternoon" if 12 <= t.hour < 18 else
                    "Night"
                    for t in times],
    "Weekday": pd.Series(selected_dates).dt.day_name(),
    "Month_name": pd.Series(selected_dates).dt.month_name(),
    "Weekdaysort": pd.Series(selected_dates).dt.weekday,
    "Monthsort": pd.Series(selected_dates).dt.month,
    "Date": pd.Series(selected_dates).dt.date,
    "Time": times,
    "week_number": pd.Series(selected_dates).dt.isocalendar().week,
    "day_of_month": pd.Series(selected_dates).dt.day
})

In [110]:
print(synthetic_data)

       hour_of_day       cash_type  money coffee_name Time_of_Day    Weekday  \
0               17            Cash   4.96       Latte   Afternoon   Thursday   
1               20            Cash   7.41    Espresso       Night     Sunday   
2               19     Credit Card  12.87   Macchiato       Night   Saturday   
3               13  Mobile Payment   7.72   Americano   Afternoon     Friday   
4               12     Credit Card   3.53  Cappuccino   Afternoon   Thursday   
...            ...             ...    ...         ...         ...        ...   
37157           11     Credit Card  14.48    Espresso     Morning  Wednesday   
37158           17     Credit Card   4.05  Cappuccino   Afternoon     Monday   
37159           20            Cash  12.15   Americano       Night     Sunday   
37160           15     Credit Card  10.16       Latte   Afternoon    Tuesday   
37161           12            Cash   7.23    Espresso   Afternoon     Friday   

      Month_name  Weekdaysort  Monthsor

In [111]:
synthetic_data.to_csv('coffee_sales.csv', index=False)